**Initialization**

The next few steps(or code snippets) initialize the colab environment for running the fast.ai course. Each line of code that added are in place to avoid any error, please do not change the position of any snippet if you do not know what they do.

Note:- You will get an error saying that the code is closing in the memory limit of 12Gb. You can click 'terminate other runtimes'.
       This error is popping up because the dataset is taking up most of the space, so don't worry
      

In [7]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

gpg: keybox '/tmp/tmps8c_zbjw/pubring.gpg' created
gpg: /tmp/tmps8c_zbjw/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
··········


In [0]:
# http://pytorch.org/
# pre installation
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch

In [33]:
!pip install fastai

In [34]:
!pip install spacy

In [35]:
!python -m spacy download en

    100% |████████████████████████████████| 37.4MB 24.7MB/s 

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')



In [0]:
#%reload_ext autoreload
#%autoreload 2
%matplotlib inline

from fastai.nlp import *
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from torchtext import vocab, data, datasets

## IMBD dataset and the sentiment classification task

The [large movie view dataset](http://ai.stanford.edu/~amaas/data/sentiment/) contains a collection of 50,000 reviews from IMDB. The dataset contains an even number of positive and negative reviews. The authors considered only highly polarized reviews. A negative review has a score ≤ 4 out of 10, and a positive review has a score ≥ 7 out of 10. Neutral reviews are not included in the dataset. The dataset is divided into training and test sets. The training set is the same 25,000 labeled reviews.

The **sentiment classification task** consists of predicting the polarity (positive or negative) of a given text.

To get the dataset, in your terminal run the following commands:

`wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz`

`gunzip aclImdb_v1.tar.gz`

`tar -xvf aclImdb_v1.tar`

In [24]:
!tar -xvzf aclImdb_v1.tar.gz -C data/

aclImdb/
aclImdb/test/
aclImdb/train/
aclImdb/test/neg/
aclImdb/test/pos/
aclImdb/train/neg/
aclImdb/train/pos/
aclImdb/train/unsup/
aclImdb/imdbEr.txt
aclImdb/imdb.vocab
aclImdb/README
aclImdb/test/labeledBow.feat
aclImdb/test/urls_neg.txt
aclImdb/test/urls_pos.txt
aclImdb/train/unsupBow.feat
aclImdb/train/labeledBow.feat
aclImdb/train/urls_neg.txt
aclImdb/train/urls_pos.txt
aclImdb/train/urls_unsup.txt
aclImdb/test/neg/127_3.txt
aclImdb/test/neg/126_4.txt
aclImdb/test/neg/125_3.txt
aclImdb/test/neg/124_2.txt
aclImdb/test/neg/123_4.txt
aclImdb/test/neg/122_4.txt
aclImdb/test/neg/121_4.txt
aclImdb/test/neg/120_2.txt
aclImdb/test/neg/119_3.txt
aclImdb/test/neg/118_1.txt
aclImdb/test/neg/117_1.txt
aclImdb/test/neg/116_4.txt
aclImdb/test/neg/115_3.txt
aclImdb/test/neg/114_2.txt
aclImdb/test/neg/113_3.txt
aclImdb/test/neg/112_2.txt
aclImdb/test/neg/111_3.txt
aclImdb/test/neg/110_1.txt
aclImdb/test/neg/109_4.txt
aclImdb/test/neg/108_2.txt
aclImdb/test/neg/107_4.txt
aclImdb/test/neg/106_3.tx

aclImdb/test/neg/1239_4.txt
aclImdb/test/neg/1238_4.txt
aclImdb/test/neg/1237_2.txt
aclImdb/test/neg/1236_2.txt
aclImdb/test/neg/1235_2.txt
aclImdb/test/neg/1234_3.txt
aclImdb/test/neg/1233_1.txt
aclImdb/test/neg/1232_1.txt
aclImdb/test/neg/1231_4.txt
aclImdb/test/neg/1230_2.txt
aclImdb/test/neg/1229_4.txt
aclImdb/test/neg/1228_4.txt
aclImdb/test/neg/1227_3.txt
aclImdb/test/neg/1226_1.txt
aclImdb/test/neg/1225_1.txt
aclImdb/test/neg/1224_3.txt
aclImdb/test/neg/1223_3.txt
aclImdb/test/neg/1222_2.txt
aclImdb/test/neg/1221_1.txt
aclImdb/test/neg/1220_3.txt
aclImdb/test/neg/1219_1.txt
aclImdb/test/neg/1218_1.txt
aclImdb/test/neg/1217_2.txt
aclImdb/test/neg/1216_1.txt
aclImdb/test/neg/1215_1.txt
aclImdb/test/neg/1214_2.txt
aclImdb/test/neg/1213_3.txt
aclImdb/test/neg/1212_4.txt
aclImdb/test/neg/1211_1.txt
aclImdb/test/neg/1210_1.txt
aclImdb/test/neg/1209_3.txt
aclImdb/test/neg/1208_3.txt
aclImdb/test/neg/1207_4.txt
aclImdb/test/neg/1206_3.txt
aclImdb/test/neg/1205_3.txt
aclImdb/test/neg/120

aclImdb/test/neg/3237_1.txt
aclImdb/test/neg/3236_1.txt
aclImdb/test/neg/3235_1.txt
aclImdb/test/neg/3234_1.txt
aclImdb/test/neg/3233_2.txt
aclImdb/test/neg/3232_4.txt
aclImdb/test/neg/3231_1.txt
aclImdb/test/neg/3230_2.txt
aclImdb/test/neg/3229_1.txt
aclImdb/test/neg/3228_1.txt
aclImdb/test/neg/3227_4.txt
aclImdb/test/neg/3226_2.txt
aclImdb/test/neg/3225_1.txt
aclImdb/test/neg/3224_3.txt
aclImdb/test/neg/3223_3.txt
aclImdb/test/neg/3222_2.txt
aclImdb/test/neg/3221_2.txt
aclImdb/test/neg/3220_3.txt
aclImdb/test/neg/3219_1.txt
aclImdb/test/neg/3218_1.txt
aclImdb/test/neg/3217_1.txt
aclImdb/test/neg/3216_1.txt
aclImdb/test/neg/3215_2.txt
aclImdb/test/neg/3214_4.txt
aclImdb/test/neg/3213_3.txt
aclImdb/test/neg/3212_4.txt
aclImdb/test/neg/3211_4.txt
aclImdb/test/neg/3210_1.txt
aclImdb/test/neg/3209_4.txt
aclImdb/test/neg/3208_4.txt
aclImdb/test/neg/3207_4.txt
aclImdb/test/neg/3206_3.txt
aclImdb/test/neg/3205_2.txt
aclImdb/test/neg/3204_2.txt
aclImdb/test/neg/3203_2.txt
aclImdb/test/neg/320

aclImdb/test/neg/4746_1.txt
aclImdb/test/neg/4745_1.txt
aclImdb/test/neg/4744_1.txt
aclImdb/test/neg/4743_4.txt
aclImdb/test/neg/4742_1.txt
aclImdb/test/neg/4741_1.txt
aclImdb/test/neg/4740_1.txt
aclImdb/test/neg/4739_4.txt
aclImdb/test/neg/4738_1.txt
aclImdb/test/neg/4737_3.txt
aclImdb/test/neg/4736_1.txt
aclImdb/test/neg/4991_3.txt
aclImdb/test/neg/4990_4.txt
aclImdb/test/neg/4989_2.txt
aclImdb/test/neg/4988_1.txt
aclImdb/test/neg/4987_4.txt
aclImdb/test/neg/4986_3.txt
aclImdb/test/neg/4985_4.txt
aclImdb/test/neg/4984_4.txt
aclImdb/test/neg/4983_1.txt
aclImdb/test/neg/4982_3.txt
aclImdb/test/neg/4981_1.txt
aclImdb/test/neg/4980_1.txt
aclImdb/test/neg/4979_1.txt
aclImdb/test/neg/4978_1.txt
aclImdb/test/neg/4977_4.txt
aclImdb/test/neg/4976_1.txt
aclImdb/test/neg/4975_3.txt
aclImdb/test/neg/4974_1.txt
aclImdb/test/neg/4973_4.txt
aclImdb/test/neg/4972_1.txt
aclImdb/test/neg/4971_1.txt
aclImdb/test/neg/4970_3.txt
aclImdb/test/neg/4969_2.txt
aclImdb/test/neg/4968_4.txt
aclImdb/test/neg/496

aclImdb/test/neg/6407_1.txt
aclImdb/test/neg/6406_4.txt
aclImdb/test/neg/6405_3.txt
aclImdb/test/neg/6404_1.txt
aclImdb/test/neg/6403_3.txt
aclImdb/test/neg/6402_2.txt
aclImdb/test/neg/6401_1.txt
aclImdb/test/neg/6400_2.txt
aclImdb/test/neg/6655_2.txt
aclImdb/test/neg/6654_3.txt
aclImdb/test/neg/6653_4.txt
aclImdb/test/neg/6652_2.txt
aclImdb/test/neg/6651_1.txt
aclImdb/test/neg/6650_1.txt
aclImdb/test/neg/6649_3.txt
aclImdb/test/neg/6648_2.txt
aclImdb/test/neg/6647_1.txt
aclImdb/test/neg/6646_1.txt
aclImdb/test/neg/6645_1.txt
aclImdb/test/neg/6644_1.txt
aclImdb/test/neg/6643_2.txt
aclImdb/test/neg/6642_2.txt
aclImdb/test/neg/6641_3.txt
aclImdb/test/neg/6640_1.txt
aclImdb/test/neg/6639_2.txt
aclImdb/test/neg/6638_1.txt
aclImdb/test/neg/6637_1.txt
aclImdb/test/neg/6636_1.txt
aclImdb/test/neg/6635_1.txt
aclImdb/test/neg/6634_1.txt
aclImdb/test/neg/6633_1.txt
aclImdb/test/neg/6632_1.txt
aclImdb/test/neg/6631_2.txt
aclImdb/test/neg/6630_1.txt
aclImdb/test/neg/6629_1.txt
aclImdb/test/neg/662

...

aclImdb/train/unsup/44975_0.txt
aclImdb/train/unsup/44974_0.txt
aclImdb/train/unsup/44973_0.txt
aclImdb/train/unsup/44972_0.txt
aclImdb/train/unsup/44971_0.txt
aclImdb/train/unsup/44970_0.txt
aclImdb/train/unsup/44969_0.txt
aclImdb/train/unsup/44968_0.txt
aclImdb/train/unsup/44967_0.txt
aclImdb/train/unsup/44966_0.txt
aclImdb/train/unsup/44965_0.txt
aclImdb/train/unsup/44964_0.txt
aclImdb/train/unsup/44963_0.txt
aclImdb/train/unsup/44962_0.txt
aclImdb/train/unsup/44961_0.txt
aclImdb/train/unsup/44960_0.txt
aclImdb/train/unsup/44959_0.txt
aclImdb/train/unsup/44958_0.txt
aclImdb/train/unsup/44957_0.txt
aclImdb/train/unsup/44956_0.txt
aclImdb/train/unsup/44955_0.txt
aclImdb/train/unsup/44954_0.txt
aclImdb/train/unsup/44953_0.txt
aclImdb/train/unsup/44952_0.txt
aclImdb/train/unsup/44951_0.txt
aclImdb/train/unsup/44950_0.txt
aclImdb/train/unsup/44949_0.txt
aclImdb/train/unsup/44948_0.txt
aclImdb/train/unsup/44947_0.txt
aclImdb/train/unsup/44946_0.txt
aclImdb/train/unsup/44945_0.txt
aclImdb/

aclImdb/train/unsup/46201_0.txt
aclImdb/train/unsup/46200_0.txt
aclImdb/train/unsup/46199_0.txt
aclImdb/train/unsup/46198_0.txt
aclImdb/train/unsup/46197_0.txt
aclImdb/train/unsup/46196_0.txt
aclImdb/train/unsup/46195_0.txt
aclImdb/train/unsup/46194_0.txt
aclImdb/train/unsup/46193_0.txt
aclImdb/train/unsup/46192_0.txt
aclImdb/train/unsup/46191_0.txt
aclImdb/train/unsup/46190_0.txt
aclImdb/train/unsup/46189_0.txt
aclImdb/train/unsup/46188_0.txt
aclImdb/train/unsup/46187_0.txt
aclImdb/train/unsup/46186_0.txt
aclImdb/train/unsup/46185_0.txt
aclImdb/train/unsup/46184_0.txt
aclImdb/train/unsup/46183_0.txt
aclImdb/train/unsup/46182_0.txt
aclImdb/train/unsup/46181_0.txt
aclImdb/train/unsup/46180_0.txt
aclImdb/train/unsup/46179_0.txt
aclImdb/train/unsup/46178_0.txt
aclImdb/train/unsup/46177_0.txt
aclImdb/train/unsup/46176_0.txt
aclImdb/train/unsup/46175_0.txt
aclImdb/train/unsup/46174_0.txt
aclImdb/train/unsup/46173_0.txt
aclImdb/train/unsup/46172_0.txt
aclImdb/train/unsup/46171_0.txt
aclImdb/

aclImdb/train/unsup/47156_0.txt
aclImdb/train/unsup/47155_0.txt
aclImdb/train/unsup/47154_0.txt
aclImdb/train/unsup/47153_0.txt
aclImdb/train/unsup/47152_0.txt
aclImdb/train/unsup/47151_0.txt
aclImdb/train/unsup/47150_0.txt
aclImdb/train/unsup/47149_0.txt
aclImdb/train/unsup/47148_0.txt
aclImdb/train/unsup/47147_0.txt
aclImdb/train/unsup/47146_0.txt
aclImdb/train/unsup/47145_0.txt
aclImdb/train/unsup/47144_0.txt
aclImdb/train/unsup/47143_0.txt
aclImdb/train/unsup/47142_0.txt
aclImdb/train/unsup/47141_0.txt
aclImdb/train/unsup/47140_0.txt
aclImdb/train/unsup/47139_0.txt
aclImdb/train/unsup/47138_0.txt
aclImdb/train/unsup/47137_0.txt
aclImdb/train/unsup/47136_0.txt
aclImdb/train/unsup/47135_0.txt
aclImdb/train/unsup/47134_0.txt
aclImdb/train/unsup/47133_0.txt
aclImdb/train/unsup/47132_0.txt
aclImdb/train/unsup/47131_0.txt
aclImdb/train/unsup/47130_0.txt
aclImdb/train/unsup/47129_0.txt
aclImdb/train/unsup/47128_0.txt
aclImdb/train/unsup/47127_0.txt
aclImdb/train/unsup/47126_0.txt
aclImdb/

aclImdb/train/unsup/48348_0.txt
aclImdb/train/unsup/48347_0.txt
aclImdb/train/unsup/48346_0.txt
aclImdb/train/unsup/48345_0.txt
aclImdb/train/unsup/48344_0.txt
aclImdb/train/unsup/48343_0.txt
aclImdb/train/unsup/48342_0.txt
aclImdb/train/unsup/48341_0.txt
aclImdb/train/unsup/48340_0.txt
aclImdb/train/unsup/48339_0.txt
aclImdb/train/unsup/48338_0.txt
aclImdb/train/unsup/48337_0.txt
aclImdb/train/unsup/48336_0.txt
aclImdb/train/unsup/48335_0.txt
aclImdb/train/unsup/48334_0.txt
aclImdb/train/unsup/48333_0.txt
aclImdb/train/unsup/48332_0.txt
aclImdb/train/unsup/48331_0.txt
aclImdb/train/unsup/48330_0.txt
aclImdb/train/unsup/48329_0.txt
aclImdb/train/unsup/48328_0.txt
aclImdb/train/unsup/48327_0.txt
aclImdb/train/unsup/48326_0.txt
aclImdb/train/unsup/48325_0.txt
aclImdb/train/unsup/48324_0.txt
aclImdb/train/unsup/48323_0.txt
aclImdb/train/unsup/48322_0.txt
aclImdb/train/unsup/48321_0.txt
aclImdb/train/unsup/48320_0.txt
aclImdb/train/unsup/48319_0.txt
aclImdb/train/unsup/48318_0.txt
aclImdb/

aclImdb/train/unsup/49338_0.txt
aclImdb/train/unsup/49337_0.txt
aclImdb/train/unsup/49336_0.txt
aclImdb/train/unsup/49335_0.txt
aclImdb/train/unsup/49334_0.txt
aclImdb/train/unsup/49333_0.txt
aclImdb/train/unsup/49332_0.txt
aclImdb/train/unsup/49331_0.txt
aclImdb/train/unsup/49330_0.txt
aclImdb/train/unsup/49329_0.txt
aclImdb/train/unsup/49328_0.txt
aclImdb/train/unsup/49327_0.txt
aclImdb/train/unsup/49326_0.txt
aclImdb/train/unsup/49325_0.txt
aclImdb/train/unsup/49324_0.txt
aclImdb/train/unsup/49323_0.txt
aclImdb/train/unsup/49322_0.txt
aclImdb/train/unsup/49321_0.txt
aclImdb/train/unsup/49320_0.txt
aclImdb/train/unsup/49319_0.txt
aclImdb/train/unsup/49318_0.txt
aclImdb/train/unsup/49317_0.txt
aclImdb/train/unsup/49316_0.txt
aclImdb/train/unsup/49315_0.txt
aclImdb/train/unsup/49314_0.txt
aclImdb/train/unsup/49313_0.txt
aclImdb/train/unsup/49312_0.txt
aclImdb/train/unsup/49311_0.txt
aclImdb/train/unsup/49310_0.txt
aclImdb/train/unsup/49309_0.txt
aclImdb/train/unsup/49308_0.txt
aclImdb/

### Tokenizing

In [0]:
sl=1000
vocab_size=200000

In [38]:
PATH='data/aclImdb/'

names = ['neg','pos']
trn,trn_y = texts_labels_from_folders(f'{PATH}train',names)
val,val_y = texts_labels_from_folders(f'{PATH}test',names)

NameError: ignored

Here is the text of the first review:

In [0]:
trn[0]

"Story of a man who has unnatural feelings for a pig. Starts out with a opening scene that is a terrific example of absurd comedy. A formal orchestra audience is turned into an insane, violent mob by the crazy chantings of it's singers. Unfortunately it stays absurd the WHOLE time with no general narrative eventually making it just too off putting. Even those from the era should be turned off. The cryptic dialogue would make Shakespeare seem easy to a third grader. On a technical level it's better than you might think with some good cinematography by future great Vilmos Zsigmond. Future stars Sally Kirkland and Frederic Forrest can be seen briefly."

In [0]:
trn_y[0]

0

[`CountVectorizer`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) converts a collection of text documents to a matrix of token counts (part of `sklearn.feature_extraction.text`). Here is how you specify parameters to the CountVectorizer. We will be working with the top 200000 unigrams, bigrams and trigrams.

In [0]:
veczr = CountVectorizer(ngram_range=(1,3), tokenizer=tokenize, max_features=vocab_size)

In the next line `fit_transform(trn)` computes the vocabulary and other hyparameters learned from the training set. It also transforms the training set. Since we have to apply the *same transformation* to your validation set, the second line uses just the method `transform(val)`. `trn_term_doc` and `val_term_doc` are sparse matrices. `trn_term_doc[i]` represents training document $i$ and it is binary (it has a $1$ for each vocabulary n-gram present in document $i$  and $0$ otherwise).

In [0]:
trn_term_doc = veczr.fit_transform(trn)
val_term_doc = veczr.transform(val)

In [0]:
trn_term_doc.shape

(25000, 200000)

In [0]:
veczr.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.int64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 1.0,
 'max_features': 200000,
 'min_df': 1,
 'ngram_range': (1, 3),
 'preprocessor': None,
 'stop_words': None,
 'strip_accents': None,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': <function fastai.nlp.tokenize>,
 'vocabulary': None}

In [0]:
# here is the vocabulary
vocab = veczr.get_feature_names()

In [0]:
vocab[50:55]

['! " and', '! " as', '! " at', '! " but', '! " for']

## Weighted Naive Bayes

Our first model is a version of logistic regression with Naive Bayes features described [here](https://www.aclweb.org/anthology/P12-2018). For every document we compute binarized features as described above. Each feature if multiplied by a log-count ratio (see below for explanation). A logitic regression model is then trained to predict sentiment.

Here is how to define **log-count ratio** for a feature $f$:

$\text{log-count ratio} = \log \frac{\text{ratio of feature $f$ in positive documents}}{\text{ratio of feature $f$ in negative documents}}$

where ratio of feature $f$ in positive documents is the number of times a positive document has a feature divided by the number of positive documents.

In [0]:
# Here is how we get a model from a bag of words
md = TextClassifierData.from_bow(trn_term_doc, trn_y, val_term_doc, val_y, sl)

In [0]:
learner = md.dotprod_nb_learner()
learner.fit(0.02, 1, wds=1e-5, cycle_len=1)



    Only loading the 'en' tokenizer.



    Only loading the 'en' tokenizer.



[ 0.      0.068   0.1223  0.9165]                             



In [0]:
learner = md.dotprod_nb_learner()
learner.fit(0.02, 1, wds=1e-6)

[ 0.      0.0581  0.1073  0.9235]                             



### unigram

Here is use `CountVectorizer` with a different set of parameters. In particular ngram_range by default is set to (1, 1)so we will get unigram features. Note that we are specifiying our own `tokenize` function.

In [0]:
veczr =  CountVectorizer(tokenizer=tokenize)
trn_term_doc = veczr.fit_transform(trn)
val_term_doc = veczr.transform(val)

Here is how to compute the $\text{log-count ratio}$ `r`. 

In [0]:
x=trn_term_doc
y=trn_y

p = x[y==1].sum(0)+1
q = x[y==0].sum(0)+1
r = np.log((p/p.sum())/(q/q.sum()))
b = np.log(len(p)/len(q))

Here is the formula for Naive Bayes.

In [0]:
pre_preds = val_term_doc @ r.T + b
preds = pre_preds.T>0
(preds==val_y).mean()

0.80740000000000001

In [0]:
pre_preds = val_term_doc.sign() @ r.T + b
preds = pre_preds.T>0
(preds==val_y).mean()

0.82623999999999997

Here is how we can fit regularized logistic regression where the features are the unigrams.

In [0]:
m = LogisticRegression(C=0.1, fit_intercept=False, dual=True)
m.fit(x, y)
preds = m.predict(val_term_doc)
(preds==val_y).mean()

0.88260000000000005

### bigram with NB features

Similar to the model before but with bigram features.

In [0]:
veczr =  CountVectorizer(ngram_range=(1,2), tokenizer=tokenize)
trn_term_doc = veczr.fit_transform(trn)
val_term_doc = veczr.transform(val)

In [0]:
y=trn_y
x=trn_term_doc.sign()
val_x = val_term_doc.sign()
p = x[y==1].sum(0)+1
q = x[y==0].sum(0)+1
r = np.log((p/p.sum())/(q/q.sum()))
b = np.log(len(p)/len(q))

Here we fit regularized logistic regression where the features are the bigrams. Bigrams are giving us 2% boost. 

In [0]:
m = LogisticRegression(C=0.1, fit_intercept=False)
m.fit(x, y);

preds = m.predict(val_x)
(preds.T==val_y).mean()

0.90271999999999997

Here is the $\text{log-count ratio}$ `r`.  

In [0]:
r

matrix([[ 0.6863,  0.6863, -0.7   , ...,  0.6863, -0.7   , -0.7   ]])

Here we fit regularized logistic regression where the features are the bigrams multiplied by the $\text{log-count ratio}$. We are getting an extra boost for the normalization. 

In [0]:
x_nb = x.multiply(r)
m = LogisticRegression(dual=True, C=1, fit_intercept=False)
m.fit(x_nb, y);

In [0]:
w = m.coef_.T
preds = (val_x_nb @ w + m.intercept_)>0
(preds.T==val_y).mean()

0.91479999999999995

This is an interpolation between Naive Bayes the regulaized logistic regression approach.

In [0]:
beta=0.25

val_x_nb = val_x.multiply(r)
w = (1-beta)*m.coef_.mean() + beta*m.coef_.T
preds = (val_x_nb @ w + m.intercept_)>0
(preds.T==val_y).mean()

0.91639999999999999

In [0]:
w2 = w.T[0]*r.A1

In [0]:
preds = (val_x @ w2 + m.intercept_)>0
(preds.T==val_y).mean()

0.91479999999999995

## References

* Baselines and Bigrams: Simple, Good Sentiment and Topic Classification. Sida Wang and Christopher D. Manning [pdf](https://www.aclweb.org/anthology/P12-2018)

### Unused helpers

In [0]:
class EzLSTM(nn.LSTM):
    def __init__(self, input_size, hidden_size, *args, **kwargs):
        super().__init__(input_size, hidden_size, *args, **kwargs)
        self.num_dirs = 2 if self.bidirectional else 1
        self.input_size = input_size
        self.hidden_size = hidden_size
        
    def forward(self, x):
        h0 = c0 = Variable(torch.zeros(self.num_dirs,x.size(1),self.hidden_size)).cuda()
        outp,_ = super().forward(x, (h0,c0))
        return outp[-1]

In [0]:
def init_wgts(m, last_l=-2):
    c = list(m.children())
    for l in c:
        if isinstance(l, nn.Embedding): 
            l.weight.data.uniform_(-0.05,0.05)
        elif isinstance(l, (nn.Linear, nn.Conv1d)):
            xavier_uniform(l.weight.data, gain=calculate_gain('relu'))
            l.bias.data.zero_()
    xavier_uniform(c[last_l].weight.data, gain=calculate_gain('linear'));

class SeqSize(nn.Sequential):
    def forward(self, x):
        for l in self.children():
            x = l(x)
            print(x.size())
        return x

### End